In [ ]:
import numpy as np
import pandas as pd
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(29)

from flask import Flask, request

In [ ]:
def construct_df(requestData):
  df = pd.DataFrame(to_array(requestData), columns=[
    'heading',
    'speed',
    'fuel_consumption',
    'wind',
    'rel_wind_dir',
    'gust',
    'waves_height',
    'rel_waves_dir',
    'waves_period',
    'wwaves_height',
    'rel_wwaves_dir',
    'wwaves_period',
    'swell1_height',
    'rel_swell1_dir',
    'swell1_period',
    'swell2_height',
    'rel_swell2_dir',
    'swell2_period',
    'ocean_current_vel',
    'rel_ocean_current_dir'
  ])

  # Delete initial row containing only zeros
  df = df.drop(0)

  return df
  

def to_array(requestData):
  data = np.zeros(20)

  for dp in requestData:
    dp_data = np.array([])
    # Define vessel heading in data point
    heading = dp['vessel']['heading']
    
    dp_data = np.append(dp_data, heading)
    dp_data = np.append(dp_data, dp['vessel']['speed'])
    dp_data = np.append(dp_data, dp['vessel']['fuelConsumption']['drift'])
    # Calculate wind speed
    dp_data = np.append(dp_data, math.sqrt(dp['weather']['windU'] ** 2 + dp['weather']['windV'] ** 2))
    # Calculate wind direction and then wind direction relative to ship heading
    dp_data = np.append(dp_data, calculate_rel_dir(
      heading,
      calculate_dir(
        dp['weather']['windU'],
        dp['weather']['windV']
      )
    ))
    dp_data = np.append(dp_data, dp['weather']['gust'])
    dp_data = np.append(dp_data, dp['marineWeather']['waves']['wavesHeight'])
    # Calculate waves direction relative to vessel heading
    dp_data = np.append(dp_data, calculate_rel_dir(
      heading,
      dp['marineWeather']['waves']['wavesDirection']
    ))
    dp_data = np.append(dp_data, dp['marineWeather']['waves']['wavesPeriod'])
    dp_data = np.append(dp_data, dp['marineWeather']['wwaves']['wwavesHeight'])
    # Calculate wind waves direction relative to vessel heading
    dp_data = np.append(dp_data, calculate_rel_dir(
      heading,
      dp['marineWeather']['wwaves']['wwavesDirection']
    ))
    dp_data = np.append(dp_data, dp['marineWeather']['wwaves']['wwavesPeriod'])
    dp_data = np.append(dp_data, dp['marineWeather']['swellWaves']['swell1Height'])
    # Calculate class 1 swell waves direction relative to vessel heading
    dp_data = np.append(dp_data, calculate_rel_dir(
      heading,
      dp['marineWeather']['swellWaves']['swell1Direction']
    ))
    dp_data = np.append(dp_data, dp['marineWeather']['swellWaves']['swell1Period'])
    dp_data = np.append(dp_data, dp['marineWeather']['swellWaves']['swell2Height'])
    # Calculate class 2 swell waves direction relative to vessel heading
    dp_data = np.append(dp_data, calculate_rel_dir(
      heading,
      dp['marineWeather']['swellWaves']['swell2Direction']
    ))
    dp_data = np.append(dp_data, dp['marineWeather']['swellWaves']['swell2Period'])
    dp_data = np.append(dp_data, dp['marineWeather']['oceanCurrentVelocity'])
    # Calculate ocean current direction relative to vessel heading
    dp_data = np.append(dp_data, calculate_rel_dir(
      heading,
      dp['marineWeather']['oceanCurrentDirection']
    ))

    data = np.vstack([data, dp_data])

  return data

def calculate_dir(u, v):
  refU = 0
  refV = 1

  cos = (refU * u + refV * v) / (math.sqrt(refU ** 2 + refV ** 2) * math.sqrt(u ** 2 + v ** 2))
  rad = math.acos(cos)
  deg = rad * (180 / math.pi)

  return deg


def calculate_rel_dir(heading, param):
  param_toward = (param + 180) % 360
  relative_param = (param_toward - heading + 360) % 360

  return relative_param

In [ ]:
def train_model(df: pd.DataFrame):
  train_set, test_set = train_test_split(df, test_size=0.2, random_state=29)

  X_train = train_set.drop('fuel_consumption', axis=1)
  y_train = train_set['fuel_consumption'].copy()

  # Scale data so that all attributes range from 0 to 1
  scaler = MinMaxScaler()
  model = scaler.fit(X_train)
  data_scaled = model.transform(X_train)

  # Train model using scaled training data
  forest_reg = RandomForestRegressor(max_features=10 , n_estimators=31, random_state=29)
  forest_reg.fit(data_scaled, y_train)

  # Evaluate model using scaled test data
  X_test = test_set.drop('fuel_consumption', axis=1)
  y_test = test_set['fuel_consumption'].copy()

  X_test_scaled = model.transform(X_test)
  predictions = forest_reg.predict(X_test_scaled)

  mse = mean_squared_error(y_test, predictions)
  rmse = np.sqrt(mse)

  print(rmse)

In [ ]:
app = Flask(__name__)

@app.route('/train', methods=['POST'])
def train():
  data = request.json
  df = construct_df(data)
  train_model(df)
  return '', 200

if __name__ == '__main__':
  app.run()